In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv()

print(find_dotenv())
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

/home/a543979/hdegis-chat-backend/.env
/home/a543979/hdegis-chat-backend/key/pjt-dev-hdegis-app-454401-bd4fac2d452b.json


In [2]:
from google import genai
from google.genai import types
import pandas as pd
from utils import UserInputProcessor

In [4]:
ELASTIC_KEYWORD_SEARCH_QUERY_GENERATOR_PROMPT = '''
You are an AI assistant specialized in generating Elasticsearch query strings. Your task is to create the most effective query string for the given user question. This query string will be used to search for relevant documents in an Elasticsearch index.

Guidelines:
1. Analyze the user's question carefully.
2. Generate ONLY a query string suitable for Elasticsearch's match query.
3. Focus on key terms and concepts from the question.
4. Include synonyms, related terms, and various word forms that might be in relevant documents:
   - Include common synonyms and closely related concepts
   - Consider different tenses of verbs (e.g., walk, walks, walked, walking)
   - Include singular and plural forms of nouns
   - Add common abbreviations or acronyms if applicable
5. Use simple Elasticsearch query string syntax if helpful (e.g., OR).
6. Do not use advanced Elasticsearch features or syntax.
7. Do not include any explanations, comments, or additional text.
8. Provide only the query string, nothing else.

For the question "What is Clickthrough Data?", we would expect a response like:
clickthrough data OR click-through data OR click through rate OR CTR OR user clicks OR ad clicks OR search engine results OR web analytics OR clicking OR clicked OR click OR clicks OR engagement OR interaction OR conversion OR traffic

Use only OR as the operator. AND is not allowed.

User Question:
{user_query}

Generate the Elasticsearch query string:
'''

In [5]:
genai_client = genai.Client(
    vertexai=True,
    project='pjt-dev-hdegis-app-454401',
    location='us-central1'
)

user_input_processor = UserInputProcessor()

In [6]:
def create_keywords(user_query):
    response = genai_client.models.generate_content(
        model='gemini-2.0-flash-001',
        contents=ELASTIC_SEARCH_QUERY_GENERATOR_PROMPT.format(user_query=user_query)
    )
    return response.text

In [9]:
df.columns

Index(['질문분류', 'comment', 'idx', '폴더 선택 (검색 범위 지정)', 'lang', 'query',
       'gt_answer', 'path_1', 'filename_1', 'page_1', 'content_1', 'path_2',
       'filename_2', 'page_2', 'content_2', 'path_3', 'filename_3', 'page_3',
       'content_3', 'path_4', 'filename_4', 'page_4', 'content_4', 'path_5',
       'filename_5', 'page_5', 'content_5', 'path_6', 'filename_6', 'page_6',
       'content_6', 'path_7', 'filename_7', 'page_7', 'content_7', 'path_8',
       'filename_8', 'page_8', 'content_8', 'path_9', 'filename_9', 'page_9',
       'content_9', 'path_10', 'filename_10', 'page_10', 'content_10',
       'path_11', 'filename_11', 'page_11', 'content_11', 'path_12',
       'filename_12', 'page_12', 'content_12', 'path_13', 'filename_13',
       'page_13', 'content_13'],
      dtype='object')

In [10]:
df = pd.read_excel("QA 성능테스트 v4.xlsx")

for i, row in df.iterrows():
    if row['질문분류'] == 'reasoning':
        continue
        
    user_query = row['query']
    print(f"[{(i+1):02d}] {user_query}")
    
    if user_input_processor.detect_language(user_query) == 'ko':
        user_query = user_input_processor.tranlate(user_query)

    keywords = create_keywords(user_query)
    keywords = [keyword.strip() for keyword in keywords.split(" OR ")]
    print(f"ㄴ keywords: {keywords}\n")    
    
    

/opt/miniforge3/envs/venv_chat_backend_test/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


[01] Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated?
ㄴ keywords: ['circuit breaker', 'breakers', 'operating method', 'operation', 'spring-operated', 'spring operated', 'hydraulic-operated', 'hydraulic operated', 'mechanism', 'requirements', 'standards', 'regulations', 'specifications', 'IEC', 'ANSI', 'IEEE', 'safety', 'performance', 'design', 'construction', 'manual', 'automatic', 'electrical', 'power system', 'switchgear']

[02] Shall the local control cubicle (LCC) be a standalone type or integral type
ㄴ keywords: ['local control cubicle', 'LCC', 'standalone', 'integral', 'type', 'local', 'control', 'cubicle', 'self-supporting', 'independent', 'attached', 'built-in', 'integrated', 'form', 'kind', 'model', 'design', 'configuration']

[03] Are there any restrictions on using Chinese or foreign products?
ㄴ keywords: ['chinese products', 'foreign products', 'imported products', 'international products', 'chi